In [ ]:

import numpy as np
import cv2
import random
import time
import os

#debug = True

def YCrCb2BGR(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

def BGR2YCrCb(image):
    return cv2.cvtColor(image, cv2.COLOR_YCrCb2BGR)

def segmentImage(anchor, blockSize=16):
    h, w = anchor.shape
    hSegments = int(h / blockSize)
    wSegments = int(w / blockSize)
    totBlocks = int(hSegments * wSegments)
    return hSegments, wSegments

def getCenter(x, y, blockSize):
    return (int(x + blockSize/2), int(y + blockSize/2))

def getAnchorSearchArea(x, y, anchor, blockSize, searchArea):
    h, w = anchor.shape
    cx, cy = getCenter(x, y, blockSize)

    sx = max(0, cx-int(blockSize/2)-searchArea) # ne asiguram ca zone de cautare este in limite
    sy = max(0, cy-int(blockSize/2)-searchArea) # coltul stanga sus a zoneu de cautare

    #taiem cadrul de ancorare in limite pentru a produce zone de cautare a ancorei
    anchorSearch = anchor[sy:min(sy+searchArea*2+blockSize, h), sx:min(sx+searchArea*2+blockSize, w)]
    return anchorSearch

def getBlockZone(p, aSearch, tBlock, blockSize):
    px, py = p 
    px, py = px-int(blockSize/2), py-int(blockSize/2)
    px, py = max(0,px), max(0,py) #
    aBlock = aSearch[py:py+blockSize, px:px+blockSize] # recuperam macroblocul din zona de cautare ancora
    try:
        assert aBlock.shape == tBlock.shape # must be same shape

    except Exception as e:
        print(e)
        print(f"ERROR - ABLOCK SHAPE: {aBlock.shape} != TBLOCK SHAPE: {tBlock.shape}")

    return aBlock

def getMAD(tBlock, aBlock):
    return np.sum(np.abs(np.subtract(tBlock, aBlock)))/(tBlock.shape[0]*tBlock.shape[1])

def getBestMatch(tBlock, aSearch, blockSize):
    step = 4
    ah, aw = aSearch.shape
    acy, acx = int(ah/2), int(aw/2) 
    minMAD = float("+inf")
    minP = None

    while step >= 1:
        p1 = (acx, acy)
        p2 = (acx+step, acy)
        p3 = (acx, acy+step)
        p4 = (acx+step, acy+step)
        p5 = (acx-step, acy)
        p6 = (acx, acy-step)
        p7 = (acx-step, acy-step)
        p8 = (acx+step, acy-step)
        p9 = (acx-step, acy+step)
        pointList = [p1,p2,p3,p4,p5,p6,p7,p8,p9] #9puncte de cautare

        for p in range(len(pointList)):
            aBlock = getBlockZone(pointList[p], aSearch, tBlock, blockSize) # get anchor macroblock
            MAD = getMAD(tBlock, aBlock) # determine MAD
            if MAD < minMAD: # store point with minimum mAD
                minMAD = MAD
                minP = pointList[p]


        step = int(step/2)

    px, py = minP # centrul blocului ancora cu MAD minim
    px, py = px - int(blockSize / 2), py - int(blockSize / 2) 
    px, py = max(0, px), max(0, py) 
    matchBlock = aSearch[py:py + blockSize, px:px + blockSize] # returnam cel mai bun macrobloc

    return matchBlock

def blockSearchBody(anchor, target, blockSize, searchArea=7):
    h, w = anchor.shape
    hSegments, wSegments = segmentImage(anchor, blockSize)

    predicted = np.ones((h, w))*255
    bcount = 0
    for y in range(0, int(hSegments*blockSize), blockSize):
        for x in range(0, int(wSegments*blockSize), blockSize):
            bcount+=1
            targetBlock = target[y:y+blockSize, x:x+blockSize] #macroblocul curent 
            anchorSearchArea = getAnchorSearchArea(x, y, anchor, blockSize, searchArea) #luam ancora zonei de cautare 
            anchorBlock = getBestMatch(targetBlock, anchorSearchArea, blockSize) #obtinem cel mal bun macrobloc de ancorare
            predicted[y:y+blockSize, x:x+blockSize] = anchorBlock #adaucam blocul ancora pentru a prezice cadrul

    cv2.imwrite("OUTPUT/predictedtestFrame.png", predicted)

    assert bcount == int(hSegments*wSegments) #verificam ca toate macroblocurile sunt verificate
    return predicted

def getResidual(target, predicted):
    return np.subtract(target, predicted)

def getReconstructTarget(residual, predicted):
    return np.add(residual, predicted)

def showImages(*kwargs): 
    for k in range(len(kwargs)):
        cv2.imshow(f"Image: {k}", k)
        cv2.waitKey(-1)

def getResidualMetric(residualFrame):
    return np.sum(np.abs(residualFrame))/(residualFrame.shape[0]*residualFrame.shape[1])

def preprocess(anchor, target, blockSize):

    if isinstance(anchor, str) and isinstance(target, str):
        anchorFrame = BGR2YCrCb(cv2.imread(anchor))[:, :, 0] # get luma component
        targetFrame = BGR2YCrCb(cv2.imread(target))[:, :, 0] # get luma component

    elif isinstance(anchor, np.ndarray) and isinstance(target, np.ndarray):
        anchorFrame = BGR2YCrCb(anchor)[:, :, 0] 
        targetFrame = BGR2YCrCb(target)[:, :, 0] 

    else:
        raise ValueError

    #Redimensionam cadrul pentru a se potrivi segmentarii
    hSegments, wSegments = segmentImage(anchorFrame, blockSize)
    anchorFrame = cv2.resize(anchorFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))
    targetFrame = cv2.resize(targetFrame, (int(wSegments*blockSize), int(hSegments*blockSize)))
    return (anchorFrame, targetFrame)

def main(anchorFrame, targetFrame, outfile="OUTPUT", saveOutput=True, blockSize = 16):

    anchorFrame, targetFrame = preprocess(anchorFrame, targetFrame, blockSize) #procese cadru

    predictedFrame = blockSearchBody(anchorFrame, targetFrame, blockSize) #cadrul prezis
    residualFrame = getResidual(targetFrame, predictedFrame) #cadrul rezidual
    naiveResidualFrame = getResidual(anchorFrame, targetFrame) #cadru rezidual fara block matching
    reconstructTargetFrame = getReconstructTarget(residualFrame, predictedFrame) #cadrul rezidual reconstruit
   # showImages(targetFrame, predictedFrame, residualFrame)

    residualMetric = getResidualMetric(residualFrame)
    naiveResidualMetric = getResidualMetric(naiveResidualFrame)

    rmText = f"Residual Metric: {residualMetric:.2f}"
    nrmText = f"Naive Residual Metric: {naiveResidualMetric:.2f}"

    isdir = os.path.isdir(outfile)
    if not isdir:
        os.mkdir(outfile)

    if saveOutput:
        cv2.imwrite(f"{outfile}/targetFrame.png", targetFrame)
        cv2.imwrite(f"{outfile}/cadruPrezis.png", predictedFrame)
        cv2.imwrite(f"{outfile}/CadruRezidual.png", residualFrame)
        cv2.imwrite(f"{outfile}/CadruTintaReconstruit.png", reconstructTargetFrame)
        cv2.imwrite(f"{outfile}/naiveResidualFrame.png", naiveResidualFrame)
        resultsFile = open(f"{outfile}/results.txt", "w"); resultsFile.write(f"{rmText}\n{nrmText}\n"); resultsFile.close()

    print(rmText)
    print(nrmText)

    return residualMetric, residualFrame

if __name__ == "__main__":
    pass
    
    anchorPath = "1.jpg"
    targetPath = "2.jpg"
    main(anchorPath, targetPath)
    

In [ ]:
def block_matching(image1, image2, block_size):
    rows1, cols1, channels1 = image1.shape
    rows2, cols2, channels2 = image2.shape
    
    # make sure the images have the same size and number of channels
    if rows1 != rows2 or cols1 != cols2 or channels1 != channels2:
        raise ValueError('Images must have the same size')
        
    matched_blocks = 0
    
    for i in range(0, rows1 - block_size + 1, block_size):
        for j in range(0, cols1 - block_size + 1, block_size):
            block1 = image1[i:i+block_size, j:j+block_size]
            block2 = image2[i:i+block_size, j:j+block_size]
            
            # compare the blocks and increment the match count if they are similar
            if (block1 == block2).all():
                matched_blocks += 1
                
    return matched_blocks


# Read the input images
image1 = cv2.imread("1.jpg")
image2 = cv2.imread("2.jpg")


# Show the displacement map
blocks_matched = block_matching(image1, image2, 5)

print('blocks matched : ',blocks_matched)